<a href="https://colab.research.google.com/github/kunwarsaaim/Image_Captioning/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from os import listdir
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Input
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import Concatenate
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import cv2


Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount("content")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at content


In [0]:
import os
os.chdir("/content/content/My Drive/Datasets/Flickr_8k")

In [0]:
def load_photos(directory,keys):      #directory:in which photos are stored
  image_arr = []
  for i in range(10):
    filename = directory + '/' + keys[i]+ '.jpg'
    #load image from file
    image = load_img(filename,target_size=(224,224))
   # image =  cv2.imread(filename)
    #image = cv2.resize(image,(224,224))
    #convert to numpy array
    image = img_to_array(image)
    #reshape data for the model
    image = preprocess_input(image)
    #apppend image to images array
    image_arr.append(image)
  image_arr = np.asarray(image_arr)
  return image_arr
                          

In [0]:
def load_doc(filename):
  #open file
  file = open(filename,'r')
  text = file.read()
  file.close()
  return text


#discription for images
def load_discription(doc):
  mapping = dict()
  for line in doc.split("\n"):
    #split line by white spaces
    tokens = line.split()
    if len(line)<2:
      continue
    #first token as image id and rest as caption
    image_id,caption = tokens[0],tokens[1:]
    #remove filename from image
    image_id = image_id.split('.')[0]
    #tokens to string
    caption = " ".join(caption)
    #store first caption
    if image_id not in mapping:
      mapping[image_id] = caption
  return mapping

    

In [0]:
def text_prepross(filename):
    #load text from file
    raw_text = load_doc(filename)
    #seprate caption and image name
    text_dict = load_discription(raw_text)
    #convert dict to dataframe
    text_df = pd.DataFrame(list(text_dict.items()))
    #convert text to integers
    keys = text_df[0].values
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text_df[1])
    vocab_size = len(tokenizer.word_index)+1
    seq = tokenizer.texts_to_sequences(text_df[1])
    #make all the tokenized caption of equal length
    padded_seq = (sequence.pad_sequences(seq))
   # text = tokenizer.sequences_to_texts(padded_seq[0])
    #make 2d array to list to assign to dataframe col
    #padded_seq = padded_seq.tolist()
#     for i in range(len(padded_seq)):
#         padded_seq[i] = np.array(padded_seq[i])
#         padded_seq[i] = padded_seq[i].reshape(1,33)
 #   text_df[2] = padded_seq
    return padded_seq,vocab_size,tokenizer,keys
    
#     b = tokenizer.sequences_to_texts([padded_seq[0]]) to covert seq to text

In [0]:
filename = "Flickr_TextData/Flickr8k.token.txt"

In [0]:
padded_seq,vocab,tokenizer,keys = text_prepross(filename)

In [0]:
image_arr = load_photos("Images",keys)

In [0]:
def model_arc(vocab_size=4421,learning_rate=0.00051,lstm_layers=3,embedding_size=300,dropout_rate=0.22):
    image_input = Input(shape=(224,224,3))
    model_vgg16 = VGG16(weights='imagenet',input_tensor=image_input)
    for layer in model_vgg16.layers:
        layer.trainable = False
    dense_input = BatchNormalization(axis=-1)(model_vgg16.layers[-2].output)
    image_dense = Dense(units=embedding_size)(dense_input)
    image_embedding = RepeatVector(1)(image_dense)
    sentence_input = Input(shape=(1,33,))
    word_embedding = Embedding(input_dim=vocab_size,output_dim=embedding_size)(sentence_input)
    sequence_input = Concatenate(axis=1)([image_embedding,word_embedding])
    input_ = sequence_input
    for _ in range(lstm_layers):
        input_ = BatchNormalization(axis=1)(input_)
        lstm_out = LSTM(units=embedding_size,
                       return_sequences=True,
                       dropout=dropout_rate,
                       recurrent_dropout=dropout_rate)(input_)
        input_ = lstm_out
    sequence_output = TimeDistributed(Dense(units=vocab_size))(lstm_out)
    
    model = Model(inputs=[image_input,sentence_input],outputs=sequence_output)
    model.compile(optimizer=Adam(lr=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
    return model
    

In [0]:
model_arc()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]          